<a href="https://colab.research.google.com/github/Alex120699/Master-IA3-2/blob/main/PerrosGatos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds

#Descargamos base de datos perros y gatos
datos, metadatos = tfds.load('cats_vs_dogs', as_supervised=True, with_info=True)

In [ ]:
#tfds.as_dataframe(datos['train'].take(5),metadatos)
#fds.show_examples(datos['train'].take(5),metadatos)

In [ ]:
import matplotlib.pyplot as plt
import cv2

#Hacemos las imagenes mas grandes
plt.figure(figsize=(20,20))

#Necesitamos que las imagenes sean del mismo tamaño todas. Cuanto mas grande sean, mas costoso, pero cuanto mas pequeño peor definicion
tamaño_img=100

n=int(input("Cuantas imagenes quieres ver: ")) #Convertimos el string en un entero
for i,(imagen,etiqueta) in enumerate(datos['train'].take(n)):

  #Convertimos las imagenes en una 200x200
  imagen=cv2.resize(imagen.numpy(),(tamaño_img,tamaño_img)) #Se convierte la imagen en una matriz con .numpy()

  #Convertimos las imagenes en blanco y negro para que pesen mucho menos
  imagen=cv2.cvtColor(imagen, cv2.COLOR_BGR2GRAY)

  #Las ordenamos en filas y columnas mediante subplot(nrows, ncols, index, **kwargs)

  plt.subplot(round(n/5)+1,5,i+1) #Las he ordenado en cinco columnas

  #Le quitamos las etiquetas de los ejes X e Y

  plt.xticks([])
  plt.yticks([])

  #Las mostramos por pantalla

  plt.imshow(imagen,cmap='gray')

In [ ]:
datos_entrenamiento=[] #Preparamos la lista
type(datos_entrenamiento)

In [ ]:
for i,(imagen,etiqueta) in enumerate(datos['train']):#Todos los datos
  imagen=cv2.resize(imagen.numpy(),(tamaño_img,tamaño_img))
  imagen=cv2.cvtColor(imagen, cv2.COLOR_BGR2GRAY)
  imagen=imagen.reshape(tamaño_img,tamaño_img,1) #Como hemos cambiado a escala de grises, indicamos que es una matriz 100x100 x 1 (si fuese rgb sera x3)
  datos_entrenamiento.append([imagen,etiqueta])

In [ ]:
print(f"datos_entrenamiento es una lista con {len(datos_entrenamiento)} imagenes")
print(f"Cada imagen es una lista con {len(datos_entrenamiento[0])} elementos")
print(f"El primer elemento de la imagen es un {type(datos_entrenamiento[0][0])} con los pixels en una matriz {datos_entrenamiento[0][0].shape}")
print(f"El segundor elemento de la imagen es un {type(datos_entrenamiento[0][1])} con 0 o 1 si gato o perro")

In [ ]:
datos_entrenamiento[0] #En resultado, numpy=1 indica que es un perro

In [ ]:
len(datos_entrenamiento) #Total de imagenes

In [ ]:
#Creamos la matriz predictores y la de respuestas (Imagenes y etiquetas)
X=[] #(pixeles)
Y=[] #Gato (0) o perro (1)

for imagen,etiqueta in datos_entrenamiento:
  X.append(imagen)
  Y.append(etiqueta)

In [ ]:
#Notamos que cada elemento de X es una matriz que representa una imagen de las 23262 que hay
import numpy as np
X=np.array(X)
Y=np.array(Y)

In [ ]:
#Divimos el dataset en 3 diferentes y lo normalizamos entre [0,1]
from sklearn.model_selection import train_test_split
#X=np.squeeze(X)
X_train_total, X_test, Y_train_total, Y_test = train_test_split(X/255., Y, test_size=0.1)
X_train, X_val, Y_train, Y_val =train_test_split(X_train_total, Y_train_total,test_size=0.2)
class_names=["gato","perro"]

In [ ]:
print(f"Total Set: {len(X)}")
print(f"Training Set: {len(X_train)}")
print(f"Validation Set: {len(X_val)}")
print(f"Testing Set: {len(X_test)}")

In [ ]:
#Vemos la estructura ahora arreglada de cada matriz
#Solo se puede plotear si tenemos cada matriz mxn y no mxnx1
(X.shape,Y.shape)
plt.imshow(X_train[0][:][:],cmap="binary")
plt.axis("off")
plt.colorbar()

In [ ]:
#Empezamos con los modelos

In [ ]:
#Modelo Denso

In [ ]:
modeloDenso=tf.keras.models.Sequential([
     tf.keras.layers.Flatten(input_shape=(100,100,1)),
     tf.keras.layers.Dense(150,activation='relu'),
     tf.keras.layers.Dense(150,activation='relu'),
     tf.keras.layers.Dense(1,activation='sigmoid')                            
])

In [ ]:
modeloDenso.compile(optimizer='adam',
                    loss='binary_crossentropy',
                    metrics=['accuracy'])

In [ ]:
#Modelo convolucional

In [ ]:
modeloCNN=tf.keras.models.Sequential([
     tf.keras.layers.Conv2D(32,(3,3),activation='relu',input_shape=(100,100,1)),
     tf.keras.layers.MaxPooling2D(2,2),
     tf.keras.layers.Conv2D(64,(3,3),activation='relu'),
     tf.keras.layers.MaxPooling2D(2,2),
     tf.keras.layers.Conv2D(128,(3,3),activation='relu'),
     tf.keras.layers.MaxPooling2D(2,2),

     tf.keras.layers.Flatten(),
     tf.keras.layers.Dense(100,activation='relu'),
     tf.keras.layers.Dense(1,activation='sigmoid')
])

In [ ]:
modeloCNN.compile(optimizer='adam',
                    loss='binary_crossentropy',
                    metrics=['accuracy'])

In [ ]:
#history = modeloDenso.fit(X_train,Y_train,epochs=100,validation_data=(X_val,Y_val))
history = modeloCNN.fit(X_train,Y_train,epochs=20,validation_data=(X_val,Y_val))

In [ ]:
import pandas as pd
df=pd.DataFrame(history.history)
df.head()

In [ ]:
df.plot(figsize=(10,7))

In [ ]:
#modeloDenso.evaluate(X_test,Y_test)
modeloCNN.evaluate(X_test,Y_test)

In [ ]:
X_new=X_test[:30]
#y_prob=modeloDenso.predict(X_new)
y_prob=modeloCNN.predict(X_new)
y_pred=np.around(y_prob,0)
y_pred

In [ ]:
fig,ax=plt.subplots(6,5,figsize=(15,20))
for i,ax in enumerate(ax.flat):
  ax.imshow(np.squeeze(X_test[i]),cmap="binary")
  ax.set_axis_off()
  ax.set_title(f"Test Name: {class_names[Y_test[i]]}\nPrediccion Name: {class_names[int(y_pred[i])]}")

In [ ]:
+